In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from nameparser import HumanName
import requests

In [88]:
team = 'PIT'
df = pd.DataFrame()

# 2018 - 2020


In [89]:
_source = {
    #2020:
}

In [90]:
employees = {
    2020: {
        
    }
}

# 2022 - Present

In [107]:
_source = {
    # 2023: 'https://web.archive.org/web/20230326092133/https://www.mlb.com/pirates/team/front-office',
    # 2022: 'https://web.archive.org/web/20220324054522/https://www.mlb.com/pirates/team/front-office',
    2021: 'https://web.archive.org/web/20210331151440/https://www.mlb.com/pirates/team/front-office',
    2020: 'https://web.archive.org/web/20200621082435/https://www.mlb.com/pirates/team/front-office'
}

In [108]:
df = pd.DataFrame()

for year in _source.keys():
    res = requests.get(_source[year])
    soup = BeautifulSoup(res.content)

    raw = [y for y in [x.text.strip().split('\n') for x in soup.select('h3, div.p-heading, div.p-table__title, tr')] if y[0] != 'Title' and y[0] != ''][3:]

    if year in [2020, 2021]:
        deps = [x.text.strip() for x in soup.select('div.p-heading')]
    else:
        deps = [x.text.strip() for x in soup.select('h3.p-heading__text')]

    department = 'Executive'
    office = None

    for _ in raw:
        if _[0].strip() in deps:
            department = _[0].title()
            office = None
        elif len(_) == 1:
            office = _[0].title()
        else:
            title = _[0]
            name = _[1]
            
            df = pd.concat([
                df, 
                pd.DataFrame({
                    'Year': year,
                    'Team': team,
                    'NameLast': [HumanName(name).last],
                    'NameFirst': [HumanName(name).first],
                    'Title': [title],
                    'Department': [department],
                    'Source of Information': [_source[year]]
                })])

df

,Year,Team,NameLast,NameFirst,Title,Department,Source of Information
0,2021,PIT,Williams,Travis,President,Executive,https://web.archive.org/web/20210331151440/htt...
0,2021,PIT,Russo,Nayli,"Vice President, Strategy",Executive,https://web.archive.org/web/20210331151440/htt...
0,2021,PIT,Trunzo,Molly,"Senior Executive Assistant, Team President",Executive,https://web.archive.org/web/20210331151440/htt...
0,2021,PIT,Cherington,Ben,General Manager,Baseball Operations,https://web.archive.org/web/20210331151440/htt...
0,2021,PIT,Stroh,Bryan,"Senior Vice President, Baseball Development",Baseball Operations,https://web.archive.org/web/20210331151440/htt...
...,...,...,...,...,...,...,...
0,2020,PIT,Wilder,David,"Clubhouse Assistant, Bradenton, FL",Florida Operations,https://web.archive.org/web/20200621082435/htt...
0,2020,PIT,Baez,Carly,Administrative Assistant,Florida Operations,https://web.archive.org/web/20200621082435/htt...
0,2020,PIT,Fogel,Spencer,"Coordinator, Florida Operations",Florida Operations,https://web.archive.org/web/20200621082435/htt...
0,2020,PIT,Quintana,Jorge,Florida Operations Assistant,Florida Operations,https://web.archive.org/web/20200621082435/htt...


In [110]:
df[['Year', 'Team', 'NameLast', 'NameFirst', 'Title', 'Department', 'Source of Information']] \
    .to_excel(f'../out/{team}-{min(_source.keys())}-to-{max(_source.keys())}.xlsx', index=False)